In [5]:
# 1. gdown 설치
!pip install -U gdown

# 2. Google Drive 파일 ID 매핑
files = {
    "scin_data.zip": "1exT7RM2JivZjXT5NVb3U61GCh4i6hjQo",
    "images_v1.zip": "1sRyIC4mw_UHzLC-5XjUuQBAfILSNQVf2",
    "images_v2.zip": "13uGHXVTyyEkiKDTSCPa2T-MFFThQ_Y-T",
    "images_v3.zip": "1jdWlsHe96cK15C51zj_PJs9C9W-l99V2",

    "matched_merged_final1_v1.zip": "1u5_dObfbysz6FXAMEvptoUS7XjefrD8z",
    "matched_merged_final1_v2.zip": "1vbxGrnQtsFENFp4tp0uy_1LfOIW2nQ6b",
    "matched_merged_final1_v3.zip": "1ZkJE4LQqi8tIsxmqqoNFbb5vzon3sN6W",
    "matched_merged_final2_v1.zip": "18RUUtGLIDbkdd-MLDpepD_U6fQhVC-Ss",
    "matched_merged_final2_v2.zip": "15zMB-a8hNlSIH_t1l2uMiJ6BhQW3t1mt",
    "matched_merged_final2_v3.zip": "1Zff-gyWALYzkGUJW0eiaPl_K_qw_q7uW",

    # https://drive.google.com/file/d/1u5_dObfbysz6FXAMEvptoUS7XjefrD8z/view?usp=drive_link
    # https://drive.google.com/file/d/1vbxGrnQtsFENFp4tp0uy_1LfOIW2nQ6b/view?usp=drive_link
    # https://drive.google.com/file/d/1ZkJE4LQqi8tIsxmqqoNFbb5vzon3sN6W/view?usp=drive_link
    # https://drive.google.com/file/d/18RUUtGLIDbkdd-MLDpepD_U6fQhVC-Ss/view?usp=drive_link
    # https://drive.google.com/file/d/15zMB-a8hNlSIH_t1l2uMiJ6BhQW3t1mt/view?usp=drive_link
    # https://drive.google.com/file/d/1Zff-gyWALYzkGUJW0eiaPl_K_qw_q7uW/view?usp=drive_link

}

# 3. 다운로드
for filename, file_id in files.items():
    !gdown --id {file_id} --output /content/{filename}
    print(f"다운로드 완료: {filename}")

# 4. 압축 해제 함수
import zipfile, os

def unzip_all():
    for filename in files.keys():
        zip_path = os.path.join("/content", filename)
        extract_dir = zip_path.replace(".zip", "")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"압축 해제 완료: {extract_dir}")

unzip_all()


/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1exT7RM2JivZjXT5NVb3U61GCh4i6hjQo
From (redirected): https://drive.google.com/uc?id=1exT7RM2JivZjXT5NVb3U61GCh4i6hjQo&confirm=t&uuid=29d9bd63-6a3d-4370-8e20-07e8579ea6e1
To: /content/scin_data.zip
100% 12.5G/12.5G [03:46<00:00, 55.2MB/s]
다운로드 완료: scin_data.zip
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may


In [6]:
# 데이터셋 파일 직접 업로드 (기본적으로 /content/ 경로에 저장)
import os

from google.colab import files
uploaded = files.upload()

filename = list(uploaded.keys())[0]
print("업로드된 파일 이름:", filename)


Saving merged_final1_v1.csv to merged_final1_v1.csv
Saving merged_final1_v2.csv to merged_final1_v2.csv
Saving merged_final1_v3.csv to merged_final1_v3.csv
Saving merged_final2_v1.csv to merged_final2_v1.csv
Saving merged_final2_v2.csv to merged_final2_v2.csv
Saving merged_final2_v3.csv to merged_final2_v3.csv
업로드된 파일 이름: merged_final1_v1.csv


In [ ]:
import os
import pandas as pd
from PIL import Image
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image

# 1. 경로 설정
csv_path = "/content/scin_data/content/scin_data/dataset/scin_cases.csv"
base_root = "/content/scin_data/content/scin_data/"

# 2. 데이터 불러오기 및 image_1_path 필터링
df = pd.read_csv(csv_path)
df = df[['case_id', 'image_1_path', 'image_2_path', 'image_3_path']]
df = df.dropna(subset=['image_1_path']).reset_index(drop=True)  # image_1_path 없는 샘플 제거

#image_2_path 및 image_3_path 존재하는 샘플 수 출력
image_2_count = df['image_2_path'].notna().sum()
image_3_count = df['image_3_path'].notna().sum()

# 3. 모든 전처리 대상 경로 수집
image_paths = []

for _, row in df.iterrows():
    for key in ['image_1_path', 'image_2_path', 'image_3_path']:
        path = row.get(key)
        if pd.notna(path):
            full_path = os.path.join(base_root, path)
            if os.path.exists(full_path):
                image_paths.append(full_path)

# 4. 중복 제거 및 정렬
image_paths = sorted(set(image_paths))

# 5. 전처리 버전 정의
transform_v1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

transform_v2 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

transform_v3 = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=20, translate=(0.1, 0.1)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# 6. 전처리 및 저장 함수
def preprocess_and_save(image_paths, transform, version_name):
    save_dir = f"/content/images_{version_name}"
    os.makedirs(save_dir, exist_ok=True)

    saved_count = 0
    for image_path in image_paths:
        try:
            img = Image.open(image_path).convert("RGB")
            transformed_tensor = transform(img)
            transformed_img = to_pil_image(transformed_tensor)

            # 원본 이미지 파일명 유지, 확장자만 .png로
            filename = os.path.splitext(os.path.basename(image_path))[0] + ".png"
            save_path = os.path.join(save_dir, filename)
            transformed_img.save(save_path)

            saved_count += 1

        except Exception as e:
            print(f"[{version_name}] {image_path} 실패 → {e}")

    print(f"\n [{version_name}] 저장 완료 → 총 {saved_count}개 파일 저장됨")
    return save_dir

# 7. 실행
dir_v1 = preprocess_and_save(image_paths, transform_v1, "v1")
dir_v2 = preprocess_and_save(image_paths, transform_v2, "v2")
dir_v3 = preprocess_and_save(image_paths, transform_v3, "v3")

# 8. 요약 정보 출력
print("\nimage_1_path 존재 샘플 수:", len(df))
print("image_2_path 존재 샘플 수:", image_2_count)
print("image_3_path 존재 샘플 수:", image_3_count)
print("전처리 대상 이미지 수 (중복 제거 후):", len(image_paths))
print("예시 이미지 경로:", image_paths[:3])



✅ [v1] 저장 완료 → 총 10379개 파일 저장됨

✅ [v2] 저장 완료 → 총 10379개 파일 저장됨

✅ [v3] 저장 완료 → 총 10379개 파일 저장됨

📦 전체 샘플 수 (image_1_path 기준): 5033
📁 전처리 대상 이미지 수 (중복 제거 후): 10379
🖼️ 예시 이미지 경로: ['/content/scin_data/content/scin_data/dataset/images/-1001492676369731180.png', '/content/scin_data/content/scin_data/dataset/images/-1001733364362669777.png', '/content/scin_data/content/scin_data/dataset/images/-1003800477193786941.png']


In [ ]:
from google.colab import files
import shutil
import os

def download_preprocessed_images():
    for version in ['v1', 'v2', 'v3']:
        folder_path = f"/content/images_{version}"
        zip_path = f"/content/images_{version}.zip"

        # 이미 zip 파일이 존재하면 삭제 (중복 방지)
        if os.path.exists(zip_path):
            os.remove(zip_path)

        # zip 파일 생성 (이름: images_v1.zip 등)
        shutil.make_archive(base_name=zip_path.replace('.zip', ''), format='zip', root_dir=folder_path)
        print(f"압축 완료: {zip_path}")

        # 로컬로 다운로드
        files.download(zip_path)

download_preprocessed_images()

📦 압축 완료: /content/images_v1.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📦 압축 완료: /content/images_v2.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📦 압축 완료: /content/images_v3.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 버전 1: .zip 압축 파일에서 압축 해제 후 메타 정보 매핑
import pandas as pd
import os
from zipfile import ZipFile

# 이미지 압축 해제 함수
def extract_images(zip_path, extract_dir):
    os.makedirs(extract_dir, exist_ok=True)
    with ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    return set(os.listdir(extract_dir))  # 이미지 파일명 리스트 반환

# 메타 정보 CSV 로드 및 파일명 추출
def load_meta(path):
    df = pd.read_csv(path)
    df['filename'] = df['image'].apply(lambda x: os.path.basename(str(x)))
    return df

# 매핑 수행
def match_images(meta_df, image_filenames):
    return meta_df[meta_df['filename'].isin(image_filenames)].copy()

# 전체 파이프라인
def generate_merged_datasets_with_extraction():
    # 메타 정보 경로
    meta_paths = {
        'final1': '/content/processed_scin_final1.csv',
        'final2': '/content/processed_scin_final2.csv'
    }

    # .zip 경로 및 압축 해제 위치
    image_versions = {
        'v1': ('/content/images_v1.zip', '/content/images_v1'),
        'v2': ('/content/images_v2.zip', '/content/images_v2'),
        'v3': ('/content/images_v3.zip', '/content/images_v3')
    }

    # 압축 해제 및 파일명 수집
    extracted_files = {}
    for version, (zip_path, folder_path) in image_versions.items():
        print(f"압축 해제 중: {zip_path}")
        extracted_files[version] = extract_images(zip_path, folder_path)

    # 매핑 수행
    for meta_key, meta_path in meta_paths.items():
        print(f"\n메타 정보 로딩 중: {meta_path}")
        meta_df = load_meta(meta_path)

        for version in image_versions:
            print(f"매핑 중... ({meta_key} + {version})")
            matched_df = match_images(meta_df, extracted_files[version])
            output_path = f"/content/merged_{meta_key}_{version}.csv"
            matched_df.to_csv(output_path, index=False)
            print(f"저장 완료: {output_path} ▶ 총 {len(matched_df)}개 샘플")
            print("메타 예시:", meta_df['filename'].unique()[:5])
            print("이미지 예시:", list(extracted_files[version])[:5])

# 실행
generate_merged_datasets_with_extraction()


In [ ]:
# 버전 2: 압축 해제된 images_v* 폴더에서 직접 매핑
import pandas as pd
import os

# 메타 정보 CSV 로드 및 파일명 추출
def load_meta(path):
    df = pd.read_csv(path)
    df['filename'] = df['image'].apply(lambda x: os.path.basename(str(x)))
    return df

# 매핑 수행
def match_images(meta_df, image_filenames):
    return meta_df[meta_df['filename'].isin(image_filenames)].copy()

# 전체 파이프라인
def generate_merged_datasets_from_existing_folders():
    # 메타 정보 경로
    meta_paths = {
        'final1': '/content/processed_scin_final1.csv',
        'final2': '/content/processed_scin_final2.csv'
    }

    # 전처리된 이미지 폴더 경로
    image_dirs = {
        'v1': '/content/images_v1',
        'v2': '/content/images_v2',
        'v3': '/content/images_v3'
    }

    # 파일명 수집
    extracted_files = {}
    for version, folder_path in image_dirs.items():
        print(f"이미지 목록 수집 중: {folder_path}")
        extracted_files[version] = set(os.listdir(folder_path))

    # 매핑 수행
    for meta_key, meta_path in meta_paths.items():
        print(f"\n메타 정보 로딩 중: {meta_path}")
        meta_df = load_meta(meta_path)

        for version in image_dirs:
            print(f"매핑 중... ({meta_key} + {version})")
            matched_df = match_images(meta_df, extracted_files[version])
            output_path = f"/content/merged_{meta_key}_{version}.csv"
            matched_df.to_csv(output_path, index=False)
            print(f"저장 완료: {output_path} ▶ 총 {len(matched_df)}개 샘플")
            print("메타 예시:", meta_df['filename'].unique()[:5])
            print("이미지 예시:", list(extracted_files[version])[:5])

# ▶ 실행
generate_merged_datasets_from_existing_folders()


📁 이미지 목록 수집 중: /content/images_v1
📁 이미지 목록 수집 중: /content/images_v2
📁 이미지 목록 수집 중: /content/images_v3

🧾 메타 정보 로딩 중: /content/processed_scin_final1.csv
🔗 매핑 중... (final1 + v1)
✅ 저장 완료: /content/merged_final1_v1.csv ▶ 총 6505개 샘플
📄 메타 예시: ['-3205742176803893704.png' '-4762289084741430925.png'
 '-4027806997035329030.png' '-3799298995660217860.png'
 '-3575683440831198879.png']
📁 이미지 예시: ['-4847644363405995542.png', '8549137165884551965.png', '6993102888764442433.png', '428271300965157171.png', '3721959455546931322.png']
🔗 매핑 중... (final1 + v2)
✅ 저장 완료: /content/merged_final1_v2.csv ▶ 총 6505개 샘플
📄 메타 예시: ['-3205742176803893704.png' '-4762289084741430925.png'
 '-4027806997035329030.png' '-3799298995660217860.png'
 '-3575683440831198879.png']
📁 이미지 예시: ['-4847644363405995542.png', '8549137165884551965.png', '6993102888764442433.png', '428271300965157171.png', '3721959455546931322.png']
🔗 매핑 중... (final1 + v3)
✅ 저장 완료: /content/merged_final1_v3.csv ▶ 총 6505개 샘플
📄 메타 예시: ['-3205742176803893704.

In [ ]:
from google.colab import files
import os

# 다운로드 대상 파일 리스트
merged_files = [
    "merged_final1_v1.csv",
    "merged_final1_v2.csv",
    "merged_final1_v3.csv",
    "merged_final2_v1.csv",
    "merged_final2_v2.csv",
    "merged_final2_v3.csv"
]

# 각 파일을 다운로드
for filename in merged_files:
    file_path = os.path.join("/content/", filename)
    if os.path.exists(file_path):
        print(f"다운로드 중: {filename}")
        files.download(file_path)
    else:
        print(f"파일 없음: {filename}")


⬇️ 다운로드 중: merged_final1_v1.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ 다운로드 중: merged_final1_v2.csv


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ 다운로드 중: merged_final2_v3.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
import pandas as pd

# 검증 대상 파일과 디렉터리 정보 정의
merged_files = [
    ("merged_final1_v1.csv", "images_v1"),
    ("merged_final1_v2.csv", "images_v2"),
    ("merged_final1_v3.csv", "images_v3"),
    ("merged_final2_v1.csv", "images_v1"),
    ("merged_final2_v2.csv", "images_v2"),
    ("merged_final2_v3.csv", "images_v3")
]

base_path = "/content/"          # merged_*.csv가 있는 경로
image_base_path = "/content"              # images_v* 폴더가 있는 경로

def validate_mapping(csv_file, image_folder):
    print(f"\n검증 시작: {csv_file} vs {image_folder}")

    # 경로 설정
    csv_path = os.path.join(base_path, csv_file)
    img_dir = os.path.join(image_base_path, image_folder)

    # 파일 존재 확인
    if not os.path.exists(csv_path):
        print(f"CSV 파일이 존재하지 않습니다: {csv_path}")
        return
    if not os.path.exists(img_dir):
        print(f"이미지 폴더가 존재하지 않습니다: {img_dir}")
        return

    # 파일 로드
    df = pd.read_csv(csv_path)
    if 'filename' not in df.columns:
        print(f"CSV에 'filename' 컬럼이 없습니다: {csv_file}")
        return

    df_filenames = set(df['filename'])
    image_files = set(os.listdir(img_dir))

    # 비교
    missing_images = df_filenames - image_files
    extra_images = image_files - df_filenames
    matched_count = len(df_filenames & image_files)

    # 결과 출력
    print(f"매핑된 파일 수: {matched_count} / {len(df)}")
    print(f"누락된 이미지 수 (CSV에는 있지만 폴더에 없음): {len(missing_images)}")
    if missing_images:
        print("예시 누락 파일:", list(missing_images)[:3])

    print(f"초과 이미지 수 (폴더에 있지만 CSV에 없음): {len(extra_images)}")
    if extra_images:
        print("예시 초과 파일:", list(extra_images)[:3])

# 전체 검증 반복
for csv_file, image_folder in merged_files:
    validate_mapping(csv_file, image_folder)



🔍 검증 시작: merged_final1_v1.csv vs images_v1
✅ 매핑된 파일 수: 6505 / 6505
❗ 누락된 이미지 수 (CSV에는 있지만 폴더에 없음): 0
⚠️ 초과 이미지 수 (폴더에 있지만 CSV에 없음): 3874
   예시 초과 파일: ['3721959455546931322.png', '812201348962502129.png', '-469937817750198048.png']

🔍 검증 시작: merged_final1_v2.csv vs images_v2
✅ 매핑된 파일 수: 6505 / 6505
❗ 누락된 이미지 수 (CSV에는 있지만 폴더에 없음): 0
⚠️ 초과 이미지 수 (폴더에 있지만 CSV에 없음): 3874
   예시 초과 파일: ['3721959455546931322.png', '812201348962502129.png', '-469937817750198048.png']

🔍 검증 시작: merged_final1_v3.csv vs images_v3
✅ 매핑된 파일 수: 6505 / 6505
❗ 누락된 이미지 수 (CSV에는 있지만 폴더에 없음): 0
⚠️ 초과 이미지 수 (폴더에 있지만 CSV에 없음): 3874
   예시 초과 파일: ['3721959455546931322.png', '812201348962502129.png', '-469937817750198048.png']

🔍 검증 시작: merged_final2_v1.csv vs images_v1
✅ 매핑된 파일 수: 5830 / 5830
❗ 누락된 이미지 수 (CSV에는 있지만 폴더에 없음): 0
⚠️ 초과 이미지 수 (폴더에 있지만 CSV에 없음): 4549
   예시 초과 파일: ['-8311841729993892354.png', '-4847644363405995542.png', '6993102888764442433.png']

🔍 검증 시작: merged_final2_v2.csv vs images_v2
✅ 매핑된 파일 수: 5830 / 5830
❗

In [7]:
import os
import pandas as pd
import shutil
from zipfile import ZipFile

# 매핑된 CSV와 이미지 폴더 매칭 목록
tasks = [
    ("merged_final1_v1.csv", "images_v1"),
    ("merged_final1_v2.csv", "images_v2"),
    ("merged_final1_v3.csv", "images_v3"),
    ("merged_final2_v1.csv", "images_v1"),
    ("merged_final2_v2.csv", "images_v2"),
    ("merged_final2_v3.csv", "images_v3"),
]

base_path = "/content"
output_zip_paths = []

for csv_file, image_folder in tasks:
    print(f"\n▶ 처리 중: {csv_file} 와 {image_folder}")

    # 경로 설정
    csv_path = os.path.join(base_path, csv_file)
    image_dir = os.path.join(base_path, image_folder)
    copy_dir = os.path.join(base_path, f"matched_{csv_file.replace('.csv', '')}")

    # CSV 로드 및 이미지 파일명 수집
    df = pd.read_csv(csv_path)
    if 'filename' not in df.columns:
        print(f"'filename' 컬럼이 없습니다: {csv_file}")
        continue
    filenames = df['filename'].dropna().unique().tolist()

    # 복사 대상 폴더 생성
    os.makedirs(copy_dir, exist_ok=True)

    # 이미지 복사
    copied = 0
    for filename in filenames:
        src_path = os.path.join(image_dir, filename)
        dst_path = os.path.join(copy_dir, filename)
        if os.path.exists(src_path):
            shutil.copy2(src_path, dst_path)
            copied += 1
        else:
            print(f"파일 없음: {filename}")

    print(f"복사 완료: {copied}개 파일 → {copy_dir}")

    # zip 압축
    zip_path = f"{copy_dir}.zip"
    shutil.make_archive(base_name=copy_dir, format='zip', root_dir=copy_dir)
    output_zip_paths.append(zip_path)
    print(f"압축 완료: {zip_path}")



▶ 처리 중: merged_final1_v1.csv 와 images_v1
복사 완료: 6505개 파일 → /content/matched_merged_final1_v1
압축 완료: /content/matched_merged_final1_v1.zip

▶ 처리 중: merged_final1_v2.csv 와 images_v2
복사 완료: 6505개 파일 → /content/matched_merged_final1_v2
압축 완료: /content/matched_merged_final1_v2.zip

▶ 처리 중: merged_final1_v3.csv 와 images_v3
복사 완료: 6505개 파일 → /content/matched_merged_final1_v3
압축 완료: /content/matched_merged_final1_v3.zip

▶ 처리 중: merged_final2_v1.csv 와 images_v1
복사 완료: 5830개 파일 → /content/matched_merged_final2_v1
압축 완료: /content/matched_merged_final2_v1.zip

▶ 처리 중: merged_final2_v2.csv 와 images_v2
복사 완료: 5830개 파일 → /content/matched_merged_final2_v2
압축 완료: /content/matched_merged_final2_v2.zip

▶ 처리 중: merged_final2_v3.csv 와 images_v3
복사 완료: 5830개 파일 → /content/matched_merged_final2_v3
압축 완료: /content/matched_merged_final2_v3.zip


In [8]:
from google.colab import files
import os

# 다운로드할 파일 목록
zip_files = [
    "matched_merged_final1_v1.zip",
    "matched_merged_final1_v2.zip",
    "matched_merged_final1_v3.zip",
    "matched_merged_final2_v1.zip",
    "matched_merged_final2_v2.zip",
    "matched_merged_final2_v3.zip"
]

# 다운로드 실행
for zip_file in zip_files:
    file_path = os.path.join("/content", zip_file)
    if os.path.exists(file_path):
        files.download(file_path)
    else:
        print(f"파일 없음: {zip_file}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>